In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# load libraries
from ipywidgets import interact, fixed, Layout
import ipywidgets as widgets

import matplotlib.pyplot as plt
import numpy as np

from time import time
from datetime import datetime
from multiprocessing import Pool
from scipy.interpolate import interp1d
from scipy.stats import skew, kurtosis
import pickle
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import plot_model

from def_strain import f_strain
from normalize import Normalize, rescale_i
import Gen_XRD as genx

In [ ]:
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['axes.titlesize'] = 10
tf.keras.mixed_precision.set_global_policy("mixed_float16")

In [ ]:
data_path = "DATA PATH HERE"
input_size = 1001
output_size = 14

raw_train = np.load(data_path + "/ndarray/train.npz")
train_data = raw_train["train_data"][:,:,np.newaxis, np.newaxis] #add 2nd dimension and channel
train_label = raw_train["train_label"][:,:]

#normalize labels
normL = Normalize(train_label, norm="MinMax")
train_label = normL.forward(train_label)

#Shuffle data
indices = np.arange(train_label.shape[0])
np.random.shuffle(indices)

train_data = train_data[indices]
train_label = train_label[indices]

print(train_data.min(), train_data.max())
print(train_label.min(), train_label.max())
print(train_data.shape, train_label.shape)

In [ ]:
## Conv net with BatchNorm
filter_length = 15
pool_length = 2
activation = "relu"
use_bias=True

cnn = keras.models.Sequential()
cnn.add(keras.layers.Conv2D(16, (filter_length, 1), strides=(2, 1), use_bias=False, padding='same', input_shape=(input_size, 1, 1)))
cnn.add(keras.layers.BatchNormalization(scale=False))
cnn.add(keras.layers.Activation(activation))
cnn.add(keras.layers.Conv2D(16, (filter_length, 1), strides=(1, 1), use_bias=use_bias, padding='same'))
cnn.add(keras.layers.BatchNormalization(scale=False))
cnn.add(keras.layers.Activation(activation))
cnn.add(keras.layers.MaxPooling2D((pool_length, 1)))

cnn.add(keras.layers.Conv2D(32, (filter_length, 1), strides=(2, 1), use_bias=use_bias, padding='same'))
cnn.add(keras.layers.BatchNormalization(scale=False))
cnn.add(keras.layers.Activation(activation))
cnn.add(keras.layers.Conv2D(32, (filter_length, 1), strides=(1, 1), use_bias=use_bias, padding='same'))
cnn.add(keras.layers.BatchNormalization(scale=False))
cnn.add(keras.layers.Activation(activation))
cnn.add(keras.layers.MaxPooling2D((pool_length, 1)))

cnn.add(keras.layers.Conv2D(64, (filter_length, 1), strides=(2, 1), use_bias=use_bias, padding='same'))
cnn.add(keras.layers.BatchNormalization(scale=False))
cnn.add(keras.layers.Activation(activation))
cnn.add(keras.layers.Conv2D(64, (filter_length, 1), strides=(1, 1), use_bias=use_bias, padding='same'))
cnn.add(keras.layers.BatchNormalization(scale=False))
cnn.add(keras.layers.Activation(activation))
cnn.add(keras.layers.MaxPooling2D((pool_length, 1)))

cnn.add(keras.layers.Conv2D(128, (filter_length, 1), strides=(2, 1), use_bias=use_bias, padding='same'))
cnn.add(keras.layers.BatchNormalization(scale=False))
cnn.add(keras.layers.Activation(activation))
cnn.add(keras.layers.Conv2D(128, (filter_length, 1), strides=(1, 1), use_bias=use_bias, padding='same'))
cnn.add(keras.layers.BatchNormalization(scale=False))
cnn.add(keras.layers.Activation(activation))
cnn.add(keras.layers.MaxPooling2D((pool_length, 1)))

cnn.add(keras.layers.Flatten())
cnn.add(keras.layers.Dense(1000, use_bias=use_bias)) #expansion / mandatory
cnn.add(keras.layers.BatchNormalization(scale=False))
cnn.add(keras.layers.Activation(activation))

cnn.add(keras.layers.Dense(100, use_bias=use_bias))
cnn.add(keras.layers.BatchNormalization(scale=False))
cnn.add(keras.layers.Activation(activation))

cnn.add(keras.layers.Dense(output_size))

cnn.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-3), loss='mse')

cnn.summary()

In [ ]:
now = datetime.now()
s_time = now.strftime("%Y-%m-%d_%H-%M-%S")
dataset = data_path.split("/")[-1]
NAME = dataset+"-{}".format(s_time)
print(NAME)

In [ ]:
callbacks_list = [
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.1,
        patience=10,
    ),
        keras.callbacks.ModelCheckpoint(
        filepath = "./checkpoints/"+ NAME + "/cnn-{epoch:02d}-{val_loss:.4f}.hdf5",
        monitor="val_loss",
        verbose=0,
        save_best_only=True,
        save_weights_only=False,
        mode="auto",
    ),
]

In [ ]:
save_path = "./checkpoints/" + NAME + "/"
try:
    os.mkdir(save_path)
    print("Created output folder:", save_path)
except:
    print("Folder already exists:", save_path)
    pass
with open(save_path + NAME + ".class", "wb") as f:
    pickle.dump(normL, f)

cnn_hist = cnn.fit(train_data, train_label,
                   epochs=5,
                   batch_size= 4096,
                   callbacks=callbacks_list,
                   validation_split = 0.1)

In [ ]:
%matplotlib notebook
fig, ax1 = plt.subplots()
ax1.semilogy(cnn_hist.history["loss"])
ax1.semilogy(cnn_hist.history["val_loss"])

ax2 = ax1.twinx()
ax2.semilogy(cnn_hist.history["lr"])

np.savetxt("./checkpoints/"+NAME+"/history.txt", np.column_stack((cnn_hist.history["loss"], cnn_hist.history["val_loss"], cnn_hist.history["lr"])))

In [ ]:
# Load test set and evaluate model
raw_test = np.load(data_path + "/ndarray/test.npz")
test_data = raw_test["test_data"][:,:, np.newaxis, np.newaxis]

test_label = raw_test["test_label"][::1]

test_label = normL.forward(test_label)

cnn.evaluate(test_data, test_label, batch_size= 4096)